# Analyze association of our phenotypes with sex and age

## Data Cleaning and Prep

In [1]:
import pandas as pd
import os
phe_dir ="~/Documents/NAS/UKBB_genetics/phe_files"
pheno="lv_height.phe"
pheno_df = pd.read_csv(os.path.join(phe_dir,pheno), names=["idx","value"],delim_whitespace=True)

We load the main dataset

In [2]:
dataset = pd.read_csv("ukb45494.csv")

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (7,12,39,40,42,43,44,45,46,198,200,220,222,224,226,228,230,232,234,236,366,367,2409,2412,2422,2425,2435,2438,2448,2451,2461,2464,2474,2477,2487,2490,2500,2503,2513,2516,2525,2526,2528,2529,2539,2542,2551,2552,2554,2555,2565,2568,2578,2581,2591,2594,2604,2607,2617,2620,2629,2630,2632,2633,2643,2646,2656,2659,2669,2672,2682,2685,2695,2698,2707,2708,2710,2711,2721,2724,2734,2737,2747,2750,2759,2760,2762,2763,2772,2773,2775,2776,2785,2786,2788,2789,2798,2799,2801,2802,2813,2820,2827,2828,2835,2836,3050,3258) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


And additional dataset

In [3]:
extra_dataset = pd.read_csv("ukb47615.csv")
extra_dataset_raw = extra_dataset

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (354,2831,2925,2927,2928,2929,2930,2931,2932,2933,2934,2935,2936,2937,2938,2939,2940,2941,2942,2943,2944,2945,2946,2947,2948,2949,2950,2957,2958,2959,2960,2961,2962,2963,2964,2965,2966,2967,2968,2969,2970,2971,2972,2974,2975,2976,2977,2978,2979,2980,2981,2982,2983,2984,2986,2988,2989,3010,3080,3082,3083,3084,3085,3086,3087,3088,3089,3090,3091,3092,3093,3094,3095,3096,3097,3099,3100,3113,3114,3115,3116,3117,3118,3119,3120,3121,3122,3123,3124,3125,3126,3127,3128,3129,3130,3131,3132,3133,3134,3135,3136,3137,3138,3139,3140,3141,3142,3143,3144,3145,3146,3147,3148,3149,3150,3151,3152,3153,3154,3156,3157,3158,3159,3160,3161,3162,3163,3164,3165,3166,3167,3168,3169,3170,3171,3172,3173,3174,3175,3176,3190,3191,3192,3193,3194,3195,3196,3197,3198,3199,3200,3201,3202,3203,3204,3205,3206,3207,3208,3209,3210,3211,3212,3213,3214,3215,3216,3217,3218,3219,3220,3221,3222,3223,3224,3225,3226,3227,3228,3229,3230,323

Exclude all participants with self reported cardiovascular disease 20002, defined instances encoded as https://biobank.ndph.ox.ac.uk/showcase/instance.cgi?id=2 range from 0-3, relevant for us are 0 and 1 bc these are non-imaging visit. We exclude self reported. Figure out what are array indices that range from 0-3

Codes of cardiovascular diseases are:
* 1065 - hypertension
* 1066 - heart/cardiac problem
* 1082 1083 1086 1425 cerebrovascular disease
* 1067 peripheral vascular disease
* 1068 venous thromboembolic disease
* 1473 high cholesterol
* 1493 other venous/lymphatic disease

In [4]:
extra_dataset = extra_dataset[~extra_dataset['20002-0.0'].isin([1065,1066,1082,1083,1086,1425,1067,1068,1473,1493])]

In [5]:
extra_dataset['20002-0.0']

0         1111.0
2         1402.0
3         1598.0
4         1464.0
5         1474.0
           ...  
502453    1465.0
502456    1309.0
502457       NaN
502458    1538.0
502460    1286.0
Name: 20002-0.0, Length: 377722, dtype: float64

Select only white european (codes for white, british and irish are 1, 1001, 1002) https://biobank.ndph.ox.ac.uk/showcase/coding.cgi?id=1001

In [6]:
dataset = dataset[ dataset['21000-0.0'].isin([1,1001,1002]) ]# white, british or irish

Keep only ids present in both datasets and save everythint to a var dataset

In [7]:
dataset_CDV = dataset[dataset['eid'].isin(extra_dataset['eid'])]

# Sex assoc

First we check if there is a correlation between sex and our pheotypes, field for the sex is 31-0.0 encoded as described here https://biobank.ndph.ox.ac.uk/showcase/coding.cgi?id=9 0 - female, 1 - male

In [8]:
gender_df = dataset_CDV[['eid','31-0.0']]

We will join phenotype dataset and gender dataset

In [9]:
joined_dataset = pd.merge(left=gender_df, right=pheno_df, left_on='eid', right_on='idx')

Finally we perform linear regression

In [10]:
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np

t test

In [11]:
women_vals = joined_dataset[joined_dataset['31-0.0']==0]['value']
men_vals = joined_dataset[joined_dataset['31-0.0']==1]['value']
m1 = np.mean(women_vals)
m2 = np.mean(men_vals)
nobs1 = len(women_vals)
nobs2 = len(men_vals)
sd1 = np.std(women_vals)
sd2 = np.std(men_vals)
stats.ttest_ind_from_stats(m1, sd1, nobs1, m2, sd2, nobs2, equal_var=True)

Ttest_indResult(statistic=19.77233165781219, pvalue=2.4460184268174126e-86)

Linear Regression

In [12]:
slope, intercept, r_value, p_value, std_err = stats.linregress(joined_dataset['31-0.0'],joined_dataset['value'])
print("BETA")
print(slope)
print("PVAL")
print(p_value)

BETA
-1.1274117162012274
PVAL
2.4849380688994523e-86


LEFT TO DO: Do the test and fix for cardiac diseases

Now loop for all phenotypes

Angle seems to have high bta

In [13]:
Phenotypes = [] 
Betas = []
PVals = []
for pheno in os.listdir("/home/dockeruser/Documents/NAS/UKBB_genetics/phe_files/"):
    pheno_df = pd.read_csv(os.path.join(phe_dir,pheno), names=["idx","value"],delim_whitespace=True)
    joined_dataset = pd.merge(left=gender_df, right=pheno_df, left_on='eid', right_on='idx')
    slope, intercept, r_value, p_value, std_err = stats.linregress(joined_dataset['31-0.0'],joined_dataset['value'])
    Phenotypes.append(pheno)
    Betas.append(slope)
    PVals.append(p_value)

In [14]:
d = {'Phenotype':Phenotypes,'Beta':Betas,'PVal':PVals}
pd.DataFrame(data=d)#.to_csv("assoc/sex.csv")

,Phenotype,Beta,PVal
0,lv_height.phe,-1.127412,2.484938e-86
1,lv_width.phe,-0.762383,2.239507e-131
2,RV_angle.phe,-3.262710,0.000000e+00
3,RV_height_ED.phe,-0.232236,4.501506e-07
4,RV_hw_ratio_ED.phe,0.018043,8.197641e-79
5,RV_width_ED.phe,-1.813166,7.137750e-176
6,hw_ratio_ED.phe,0.027325,7.869632e-43
7,mitral_distance.phe,1.277932,4.424452e-275
8,lv_angle.phe,3.198152,0.000000e+00
9,max_chin_ED.phe,0.132824,3.873115e-74


# Age assoc

We check if there is a correlation between age and our phenotypes, age is given as the field 21003 - age when attended the assesment center ( our phenotypes are extracted only from the first visit MRI)

In [15]:
age_df = dataset_CDV[['eid','21003-0.0','31-0.0']]

We join datasets for age and phenotype and loop

In [16]:
Phenotypes = [] 
Betas = []
PVals = []
for pheno in os.listdir("/home/dockeruser/Documents/NAS/UKBB_genetics/phe_files/"):
    pheno_df = pd.read_csv(os.path.join(phe_dir,pheno), names=["idx","value"],delim_whitespace=True)
    joined_dataset = pd.merge(left=age_df, right=pheno_df, left_on='eid', right_on='idx')
    slope, intercept, r_value, p_value, std_err = stats.linregress(joined_dataset['21003-0.0'],joined_dataset['value'])
    Phenotypes.append(pheno)
    Betas.append(slope)
    PVals.append(p_value)

In [17]:
d = {'Phenotype':Phenotypes,'Beta':Betas,'PVal':PVals}
pd.DataFrame(data=d)#.to_csv("assoc/age.csv")

,Phenotype,Beta,PVal
0,lv_height.phe,-0.051094,2.982080e-40
1,lv_width.phe,0.011900,1.652274e-08
2,RV_angle.phe,0.041979,5.356234e-19
3,RV_height_ED.phe,0.063132,4.490855e-94
4,RV_hw_ratio_ED.phe,0.001887,1.696374e-191
5,RV_width_ED.phe,-0.040049,1.880390e-20
6,hw_ratio_ED.phe,-0.002559,3.770482e-82
7,mitral_distance.phe,0.024304,8.462391e-24
8,lv_angle.phe,-0.037600,4.184953e-15
9,max_chin_ED.phe,0.001271,8.949034e-03


## Sex X Age assoc

In [20]:
pheno_df = pd.read_csv(os.path.join(phe_dir,pheno), names=["idx","value"],delim_whitespace=True)
joined_dataset = pd.merge(left=age_df, right=pheno_df, left_on='eid', right_on='idx')
joined_dataset.rename(columns={'21003-0.0':'age','31-0.0':'sex'})
joined_dataset

,eid,21003-0.0,31-0.0,idx,value
0,1000012,59.0,0.0,1000012,0.861275
1,1000158,44.0,1.0,1000158,0.968287
2,1000243,59.0,1.0,1000243,1.053905
3,1000816,66.0,0.0,1000816,0.910319
4,1001525,62.0,1.0,1001525,0.878318
...,...,...,...,...,...
15834,6022897,55.0,1.0,6022897,0.782276
15835,6023580,61.0,0.0,6023580,0.850535
15836,6023670,58.0,1.0,6023670,0.757203
15837,6023999,59.0,0.0,6023999,0.823875


Call R script which is gonna write this to csv and produce nice plots

# Association with common cardiac diseases

Codes clinical outcomes are:
* 1065 - hypertension
* 1066 - heart/cardiac problem
* 1082 1083 1086 1425 cerebrovascular disease
* 1067 peripheral vascular disease
* 1068 venous thromboembolic disease
* 1473 high cholesterol
* 1493 other venous/lymphatic disease
* 1220 diabetes
* 1081 stroke
* 1111 asthma 
* 1112 chronic obstructive pulmonary disease copd
* 1412 bronchitis
* 1262 parkinsons disease 
* 1263 dementia
* 1286 depression

In [52]:
names = ["hypertension", "heart problem", "cerebrovascular disease", "peripheral vasccular disease","venous thromboembolic disease","high cholesterol", "lymphatic diseases","diabetes","stroke","asthma","copd","bronchitis","parkinsons disease","dementia","depression"]
codes = [[1065],[1066],[1082,1083,1086,1425],[1067],[1068],[1473],[1493],[1220],[1081],[1111],[1112],[1412],[1262],[1263],[1286]]

In [53]:
diseases = {}
for code,name in zip(codes,names):
    not_diagnosed = extra_dataset_raw[~extra_dataset_raw['20002-0.0'].isin(code)]
    diagnosed = extra_dataset_raw[extra_dataset_raw['20002-0.0'].isin(code)]
    not_diagnosed = not_diagnosed['eid']
    diagnosed = diagnosed['eid']
    phenotypes = {}
    for pheno in os.listdir("/home/dockeruser/Documents/NAS/UKBB_genetics/phe_files/"):
        try:
            print(pheno)
            pheno_df = pd.read_csv(os.path.join(phe_dir,pheno), names=["idx","value"],delim_whitespace=True)    
            not_diagnosed_joined = pd.merge(left=not_diagnosed, right=pheno_df, left_on='eid', right_on='idx')
            diagnosed_joined = pd.merge(left=diagnosed, right=pheno_df, left_on='eid', right_on='idx')
            l1 = not_diagnosed_joined['value']
            l2 = diagnosed_joined['value']
            m1 = np.mean(l1)
            m2 = np.mean(l2)
            nobs1 = len(l1)
            nobs2 = len(l2)
            sd1 = np.std(l1)
            sd2 = np.std(l2)
            pval = stats.ttest_ind_from_stats(m1, sd1, nobs1, m2, sd2, nobs2, equal_var=True).pvalue
            phenotypes[pheno.split('.')[0]] = pval
        except:
            print("no phenos for this one")
    diseases[name]=phenotypes

lv_height.phe
lv_width.phe
RV_angle.phe
RV_height_ED.phe
RV_hw_ratio_ED.phe
RV_width_ED.phe
hw_ratio_ED.phe
mitral_distance.phe
lv_angle.phe
max_chin_ED.phe
max_chin_ES.phe
max_comp.phe
median_chin_ED.phe
median_chin_ES.phe
median_comp.phe
slice5_chin_ED.phe
slice5_chin_ES.phe
slice5_comp.phe
lv_height.phe
lv_width.phe
RV_angle.phe
RV_height_ED.phe
RV_hw_ratio_ED.phe
RV_width_ED.phe
hw_ratio_ED.phe
mitral_distance.phe
lv_angle.phe
max_chin_ED.phe
max_chin_ES.phe
max_comp.phe
median_chin_ED.phe
median_chin_ES.phe
median_comp.phe
slice5_chin_ED.phe
slice5_chin_ES.phe
slice5_comp.phe
lv_height.phe
lv_width.phe
RV_angle.phe
RV_height_ED.phe
RV_hw_ratio_ED.phe
RV_width_ED.phe
hw_ratio_ED.phe
mitral_distance.phe
lv_angle.phe
max_chin_ED.phe
max_chin_ES.phe
max_comp.phe
median_chin_ED.phe
median_chin_ES.phe
median_comp.phe
slice5_chin_ED.phe
slice5_chin_ES.phe
slice5_comp.phe
lv_height.phe
lv_width.phe
RV_angle.phe
RV_height_ED.phe
RV_hw_ratio_ED.phe
RV_width_ED.phe
hw_ratio_ED.phe
mitral_dis

In [62]:
diseases

{'hypertension': {'lv_height': 9.299173855074383e-93,
  'lv_width': 7.888305712219118e-36,
  'RV_angle': 1.3102078970181883e-73,
  'RV_height_ED': 2.8218065238361955e-26,
  'RV_hw_ratio_ED': 0.011924462963082924,
  'RV_width_ED': 5.164791543746335e-52,
  'hw_ratio_ED': 4.444325709827811e-13,
  'mitral_distance': 9.965601571245042e-05,
  'lv_angle': 1.284351449090543e-08,
  'max_chin_ED': 1.3870740478715603e-12,
  'max_chin_ES': 2.1456728326407498e-05,
  'max_comp': 1.3435672733839197e-07,
  'median_chin_ED': 5.8583798824964454e-30,
  'median_chin_ES': 4.182840317479659e-12,
  'median_comp': 0.006387393451179928,
  'slice5_chin_ED': 6.170102187428792e-10,
  'slice5_chin_ES': 2.1778091283159512e-08,
  'slice5_comp': 0.023879691415891274},
 'heart problem': {'lv_height': 0.2925117812892118,
  'lv_width': 0.001684733703554774,
  'RV_angle': 0.3597067290578322,
  'RV_height_ED': 0.0013141492690305469,
  'RV_hw_ratio_ED': 0.02353802876443346,
  'RV_width_ED': 0.18030633355544723,
  'hw_ratio

In [54]:
djf = pd.DataFrame(diseases)

In [58]:
djf=djf.drop(['venous thromboembolic disease','lymphatic diseases','dementia'],axis=1)

In [59]:
djf.to_csv("assoc/cardiac_diseases.csv")

In [61]:
djf.to_pickle("dummy.pkl")